In [1]:
import matplotlib.pyplot as plt
import control as co  # pip install control
import numpy as np  
import sympy as sp

from numpy import cos, sin, pi

Given values

In [47]:
r1, s0, s1, a01 = sp.symbols('r_1, s_0, s_1, a_0_1')
ao, a1, a2, b0 = sp.symbols('a_o,\hat{a}_1,\hat{a}_2,\hat{b}_0')
a, b, c, d, q, s, Ts = sp.symbols('a, b, c, d, q, s, T')
yddot, ydot = sp.symbols('\ddot{y},\dot{y}')
y, u, uc = sp.symbols('y, u, u_c')
a_true, b_true, c_true, d_true = (1, 1, 2, 1/2)
G1 = (b/(s + a)).subs(a, 1).subs(b, 1)
G2 = c/(s + d)
G_sym = b0/(s**2 + a1*s + a2)
G = sp.Mul(G1, G2)
Bm = 1
Am = s ** 2 + 2*s + 1

In [3]:
G = sp.collect(sp.expand(G), s)
G

c/(d + s**2 + s*(d + 1))

In [4]:
B, A = sp.fraction(G)
B_plus = 1
B_minus = c

So,

$ A = $ {{A}}

and, 

$ B = $ {{B}}

In [5]:
G_sym

\hat{b}_0/(\hat{a}_1*s + \hat{a}_2 + s**2)

a) Design control parameters using the Diophantine equation

Using the compatability conditions: 
$$\text{degA}_m=\text{degA}=2$$
$$\text{degB}_m=\text{degB}=0$$
$$\text{degA}_0=\text{degA - degB}^+-1=1$$

The causality condition gives, 
$$\text{degA}_m - \text{degB}_m'\geq\text{degA} - \text{degB}^+$$

Given that $$\text{degA}_m=\text{degA}=2$$ and $$\text{degB}_m'=0$$ then $$\text{degB}^+=0$$

With this we can find, 
$$\text{degA}_c = \text{degA}_o + \text{degA}_m + \text{degB}^+ = 3$$

It is then found that, 
$$\text{degR} = \text{degA}_c - \text{degA} = 3 - 2 = 1$$
Using the minimum phase relationship, 
$$\text{degR}=\text{degT}=\text{degS}=1$$

The control parameters may then be found using the Diophantine equation, 
$$AR' + B^-S = A_0A_m$$
$$(s^2 + (a + d)s + ad)(s + r_1) + bc(s_0s + s_1) = (s + a_{o_1})(s^2 + 2s + 1)$$

In [48]:
R = s + r1
R_prime = R
S = s0*s + s1
Ao = (s + ao)
T = Ao*Bm/B_minus

$ A_m = $ {{Am}}

$ B_m = $ {{Bm}}

$ R = $ {{R}}

$ S = $ {{S}}

$ T = $ {{T}}

In [7]:
dio_LHS = sp.collect(sp.expand(A*R_prime + B_minus*S), s)
dio_RHS = sp.collect(sp.expand(Ao*Am), s)
dio_poly = sp.Poly(sp.collect(sp.expand(dio_LHS - dio_RHS), s), s)

var = [r1, s0, s1]
control_vars = []
for idx in range(len(dio_poly.coeffs())):
    control_vars.append(sp.solve(dio_poly.coeffs()[idx], var[idx])[0])
r1c_sym = control_vars[0]
s0c_sym = control_vars[1]
s1c_sym = control_vars[2]

$ r_1 = $ {{r1c_sym}}

$ s_0 = $ {{s0c_sym}}

$ s_1 = $ {{s1c_sym}}

The process is described by, 

$ p^2y(t) + (d + 1)py(t) + dy(t) = cu(t) $

Then we can define, 

$ y_f(s) = H_f(s)Y(s) $

and 

$ u_f(s) = H_f(s)U(s) $

where 

$ H_f(s) = \frac{1}{Am(s)} $

The model then becomes, 

$ \frac{p^2}{A_m(s)}y_f(t) = -(d + 1)\frac{p}{A_m(s)}y_f(t) - \frac{d}{A_m(s)}y_f(t) + \frac{c}{A_m(s)}u_f(t) $

The RLS can be used on this model by defining, 

$ y_i(t) = \frac{p^i}{Am(p)}y(t) $ 

$ u_i(t) = \frac{p^i}{Am(p)}u(t) $

With this choice of definition, we can reparameterize the model as, 

$ y_n(t) = \phi(t)^T\theta $

where, 

$ \phi(t) = \big[-y_{f1}(t)\ -y_{f0}(t)\ u_{f0}(t)\big]^T $

$ \theta =  \big[(d + 1)\ d\ c\big]^T$

To begin to evaluate the parametesr, the continuous time variables are estimated with the bilinear transformation

In [8]:
bilinear = 2*(1-q**(-1))/(T*(1+q**(-1)))


Hf1 = (s/Am).subs(s, bilinear) 
Hf1 = sp.collect(sp.simplify(sp.expand(Hf1)), q)

Hf0 = (1/Am).subs(s, bilinear) 
Hf0 = sp.collect(sp.simplify(sp.expand(Hf0)), q)

In [10]:
bf1, af1 = sp.fraction(Hf1)
af1_poly = sp.Poly(af1, q)
bf1_poly = sp.Poly(bf1, q)
af1_coeffs = []
bf1_coeffs = []
for n in range(len(af1_poly.coeffs())):
    af1_coeffs.append(sp.simplify(af1_poly.coeffs()[n]/af1_poly.coeffs()[0]))
for n in range(len(bf1_poly.coeffs())):
    bf1_coeffs.append(sp.simplify(bf1_poly.coeffs()[n]/af1_poly.coeffs()[0]))
bf1_coeffs.insert(1, 0)

In [13]:
af1_coeffs

[1, 2*(T - 2)/(T + 2), (T**2 - 4*T + 4)/(T**2 + 4*T + 4)]

In [12]:
bf1_coeffs

[2*T/(T**2 + 4*T + 4), 0, -2*T/(T**2 + 4*T + 4)]

In [37]:
bf0, af0 = sp.fraction(Hf0)
af0_poly = sp.Poly(af0, q)
bf0_poly = sp.Poly(bf0, q)
af0_coeffs = []
bf0_coeffs = []
for n in range(len(af0_poly.coeffs())):
    af0_coeffs.append(sp.simplify(af0_poly.coeffs()[n]/af0_poly.coeffs()[0]))
for n in range(len(bf0_poly.coeffs())):
    bf0_coeffs.append(sp.simplify(bf0_poly.coeffs()[n]/af0_poly.coeffs()[0]))

In [38]:
af0_coeffs

[1, 2*(T - 2)/(T + 2), (T**2 - 4*T + 4)/(T**2 + 4*T + 4)]

In [43]:
bf0_coeffs

[T**2/(T**2 + 4*T + 4), 2*T**2/(T**2 + 4*T + 4), T**2/(T**2 + 4*T + 4)]